## Looping through all patients

In [ ]:
# Avoid repeating any RIDS
unique_rids = patient_data['RID'].unique()
unique_rids = [21, 31, 56]

# Create a data frame to collect results for each patient
results = pandas.DataFrame(columns = ['RID','Model','R squared','alpha','beta','gamma','s','p','b','k'])

# Loop through all patients 
for rid in unique_rids: 
    
    patient = patient_data[patient_data['RID'] == rid] 

    # Define number of time points 
    num_time_points = patient.shape[0]

    # Define list with stage assingments for each time point
    stages = patient['ml_stage'].tolist()
    stages = [int(x) for x in stages]

    # Define patient tau data
    patient_tau = patient.iloc[:,3:]

    # Convert to array and transpose so dim = nROI x # of stages
    patient_tau = patient_tau.to_numpy()
    patient_tau = patient_tau.T
    patient_tau_reshaped = patient_tau.reshape(-1)

    # Run Nexis global and calculate R squared
    custom_Nexis_error_global = partial(Nexis_error, patient_tau=patient_tau, stages=stages, nexis_model=nexis_model_global)
    result = minimize(custom_Nexis_error_global, init_guess_global, bounds=bounds_global, method='L-BFGS-B', 
                    options={'ftol': 1e-6, 'gtol': 1e-6})
    if result.success:
        optimized_params_global = result.x
        optimized_params_global = optimized_params_global.tolist( )
    Y = nexis_model_global.simulate_nexis(optimized_params_global)
    Y_edited = Y[:, stages]
    Y_edited_reshaped = Y_edited.reshape(-1)
    R_global, p_value_global = pearsonr(patient_tau_reshaped, Y_edited_reshaped)
    R_squared_global = R_global ** 2

    # Run Nexis with gene and calculate R squared
    custom_Nexis_error_gene = partial(Nexis_error, patient_tau=patient_tau, stages=stages, nexis_model=nexis_model_gene)
    result = minimize(custom_Nexis_error_gene, init_guess_gene, bounds=bounds_gene, method='L-BFGS-B', 
                    options={'ftol': 1e-6, 'gtol': 1e-6})
    if result.success:
        optimized_params_gene = result.x
        optimized_params_gene = optimized_params_gene.tolist()
    Y = nexis_model_gene.simulate_nexis(optimized_params_gene)
    Y_edited = Y[:, stages]
    Y_edited_reshaped = Y_edited.reshape(-1)
    R_gene, p_value_gene = pearsonr(patient_tau_reshaped, Y_edited_reshaped)
    R_squared_gene = R_gene ** 2

    # Add to data frame
    new_rows = pandas.DataFrame({'RID': [rid, rid], 'Model':['Global','Gene'], 
                                 'R squared': [R_squared_global, R_squared_gene],
                                'alpha': [optimized_params_global[0], optimized_params_gene[0]],
                                'beta': [optimized_params_global[1], optimized_params_gene[1]],
                                'gamma': [optimized_params_global[2], optimized_params_gene[2]],
                                's':[optimized_params_global[3], optimized_params_gene[3]],
                                'b':[optimized_params_global[4], optimized_params_gene[4]],
                                'p':[optimized_params_global[5], optimized_params_gene[5]],
                                'k':[optimized_params_global[6], optimized_params_gene[6]]})
    
    results = pandas.concat([results, new_rows], ignore_index=True)


# Print data frame
results